In [1]:
#you can use %
%pip install -q llama-index-core llama-index-readers-file llama-index-llms-nvidia llama-index-utils-workflow

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
from llama_index.core.workflow import Workflow, step, Event, Context

from llama_index.core.workflow.events import (
    StartEvent,
    StopEvent,
    InputRequiredEvent,
    HumanResponseEvent
)

# some event types to define the workflow:

# if the user says the research is not good enough, we retry the workflow
class RetryEvent(Event):
    text: str
    feedback: str

class DownloadEvent(Event):
    audio_file:str

class DownloadEvent(Event):
    audio_file:str

# if the user says the research is good enough, we generate a report
class OptimizeEvent(Event):
     text: str

# we emit progress events to the frontend so the user knows what's happening
class ValidateEvent(Event):
    valid: bool

In [7]:
from llama_index.llms.nvidia import NVIDIA
import os
from transcribe import download_audio, transcribe


if os.getenv('NVIDIA_API_KEY') is None:
    raise ValueError("NVIDIA_API_KEY environment variable is not set")

class OptimizeWorflow(Workflow):
    llm = NVIDIA(model="meta/llama3-70b-instruct")
    
    @step
    async def download(self, ctx: Context, ev: StartEvent) -> DownloadEvent:
        # ctx.write_event_to_stream(ProgressEvent(msg=f"I am doing some research on the subject of '{ev.query}'"))
        # await ctx.set("original_query", ev.query)

        if ev.query is None:
            url = "https://www.youtube.com/watch?v=dyBLJqwO1Dc"
        else:
            url = ev.query

        audio_file = download_audio(url)

        # once we've done the research, we send what we've found back to the human for review
        # this gets handled by the frontend, and we expect a HumanResponseEvent to be sent back
        return DownloadEvent(prefix="", audio_file=audio_file)
    

    @step
    async def transcribe(self, ctx: Context, ev: DownloadEvent) -> OptimizeEvent:
        # ctx.write_event_to_stream(ProgressEvent(msg=f"I am doing some research on the subject of '{ev.query}'"))
        # await ctx.set("original_query", ev.query)

        # once we've done the research, we send what we've found back to the human for review
        # this gets handled by the frontend, and we expect a HumanResponseEvent to be sent back
        return OptimizeEvent(prefix="", query=ev.query,payload=f"Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed ut purus eget sapien. Nulla facilisi. Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.")
    

/Users/iolanigan/github/ar-transliterate/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
